In [1]:
import random
import os

from neat import DistributedEvaluator, ParallelEvaluator
from tqdm.notebook import tqdm, trange
from pathlib import Path
from random import randint
from ui.python.Layout import Layout
import numpy as np

from helpers.estimation_helpers import *
import helpers.visualize as visualize

In [2]:
import pandas as pd

In [3]:
MAX_WORKERS = 24
SLICE_SIZE = 1000
EPOCHS = 40
SCORE_COEFFICIENTS = (1000, 0, 0)

# Preprocessing

In [4]:
df = pd.read_csv('./../data/datasets/ECommerce_consumer behaviour.csv')
df

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,department,product_name
0,2425083,49125,1,2,18,NaN,17,1,0,13,pantry,baking ingredients
1,2425083,49125,1,2,18,NaN,91,2,0,16,dairy eggs,soy lactosefree
2,2425083,49125,1,2,18,NaN,36,3,0,16,dairy eggs,butter
3,2425083,49125,1,2,18,NaN,83,4,0,4,produce,fresh vegetables
4,2425083,49125,1,2,18,NaN,83,5,0,4,produce,fresh vegetables
...,...,...,...,...,...,...,...,...,...,...,...,...
2019496,3390742,199430,16,3,18,5.0,83,8,0,4,produce,fresh vegetables
2019497,458285,128787,42,2,19,3.0,115,1,1,7,beverages,water seltzer sparkling water
2019498,458285,128787,42,2,19,3.0,32,2,1,4,produce,packaged produce
2019499,458285,128787,42,2,19,3.0,32,3,1,4,produce,packaged produce


In [5]:
df = df[['order_id', 'user_id', 'order_number', 'department', 'product_id', 'product_name']]
df

,order_id,user_id,order_number,department,product_id,product_name
0,2425083,49125,1,pantry,17,baking ingredients
1,2425083,49125,1,dairy eggs,91,soy lactosefree
2,2425083,49125,1,dairy eggs,36,butter
3,2425083,49125,1,produce,83,fresh vegetables
4,2425083,49125,1,produce,83,fresh vegetables
...,...,...,...,...,...,...
2019496,3390742,199430,16,produce,83,fresh vegetables
2019497,458285,128787,42,beverages,115,water seltzer sparkling water
2019498,458285,128787,42,produce,32,packaged produce
2019499,458285,128787,42,produce,32,packaged produce


In [6]:
df['product_name'].unique()

array(['baking ingredients', 'soy lactosefree', 'butter',
       'fresh vegetables', 'yogurt', 'canned meals beans',
       'poultry counter', 'ice cream ice', 'fresh fruits', 'milk',
       'packaged cheese', 'bread', 'tea', 'bakery desserts',
       'frozen breakfast', 'cereal', 'eggs', 'buns rolls', 'cream',
       'water seltzer sparkling water', 'pickled goods olives',
       'packaged poultry', 'other creams cheeses',
       'honeys syrups nectars', 'coffee', 'refrigerated',
       'energy granola bars', 'soft drinks', 'latino foods',
       'plates bowls cups flatware', 'paper goods', 'oral hygiene',
       'diapers wipes', 'food storage', 'nuts seeds dried fruit', 'soap',
       'packaged vegetables fruits', 'hot dogs bacon sausage',
       'lunch meat', 'chips pretzels', 'meat counter',
       'fresh dips tapenades', 'prepared soups salads', 'condiments',
       'juice nectars', 'canned fruit applesauce',
       'preserved dips spreads', 'packaged produce',
       'canned jarr

In [7]:
df['department'].unique()

array(['pantry', 'dairy eggs', 'produce', 'canned goods', 'meat seafood',
       'frozen', 'bakery', 'beverages', 'breakfast', 'snacks',
       'international', 'household', 'personal care', 'babies', 'deli',
       'dry goods pasta', 'missing', 'alcohol', 'pets', 'bulk', 'other'],
      dtype=object)

In [8]:
item_list = df['product_id'].unique().tolist()
item_list

[17,
 91,
 36,
 83,
 120,
 59,
 35,
 37,
 24,
 84,
 21,
 112,
 94,
 8,
 52,
 121,
 86,
 43,
 53,
 115,
 110,
 49,
 108,
 29,
 26,
 31,
 3,
 77,
 30,
 111,
 54,
 20,
 56,
 85,
 117,
 25,
 123,
 106,
 96,
 107,
 122,
 67,
 1,
 72,
 98,
 99,
 51,
 32,
 81,
 12,
 9,
 116,
 129,
 69,
 131,
 13,
 16,
 130,
 104,
 63,
 58,
 100,
 23,
 57,
 133,
 64,
 78,
 45,
 50,
 128,
 14,
 27,
 75,
 66,
 34,
 38,
 88,
 46,
 11,
 93,
 125,
 101,
 126,
 48,
 4,
 124,
 89,
 105,
 19,
 92,
 44,
 40,
 82,
 79,
 5,
 42,
 55,
 134,
 61,
 114,
 15,
 68,
 119,
 109,
 80,
 62,
 65,
 95,
 70,
 60,
 71,
 2,
 18,
 28,
 127,
 22,
 47,
 90,
 118,
 74,
 7,
 6,
 41,
 76,
 97,
 39,
 103,
 102,
 87,
 33,
 73,
 10,
 132,
 113]

In [9]:
# return tuple {orderId, list(str(items_ids))} for single check
def get_order_items(order_id):
    return order_id, list(map(str, df[df['order_id'] == order_id]['product_id'].tolist()))

In [10]:
check_ids = df['order_id'].unique().tolist()
check_list = []
for check_id in tqdm(check_ids[:2000]):
    check_list.append(get_order_items(check_id))

  0%|          | 0/2000 [00:00<?, ?it/s]

# Layout functions

In [11]:
def random_layout():
    layout = Layout('./../data/layout 18x25_6.json')
    layout.set_item_list(df['product_id'].unique().tolist())
    for row in range(layout.shape[0]):
        for col in range(layout.shape[1]):
            if layout[row][col].type.name == 'RACK':
                for lev in range(layout.get_max_rack_level()):
                    layout.set_item_to_rack(random.choice(layout.get_item_list()), (row, col), level=lev)
    return layout

In [12]:
layout = Layout('./../data/layout 18x25_6.json').get_empty_rack_layout()
items = df['product_id'].unique().tolist()
str_item = [str(i) for i in items]
layout.set_item_list(str_item)

In [14]:
def create_input_for_genome(layout, i, j, level):
    def tile_enum_to_int(tile):
        if tile.type.value == 'wall':
            return 0
        if tile.type.value == 'floor':
            return 1
        if tile.type.value == 'rack':
            return 2
        if tile.type.value == 'door':
            return 3
        if tile.type.value == 'cashier':
            return 4
        return 0
    
    def convert_items_to_int(items):
        ids = []
        if len(items) == 0:
            return [(-1, -1), (-1, -1), (-1, -1), (-1, -1)]
        for item in items:
            if item[0] == '':
                ids.append((-1, 0))
            else:
                ids.append((int(item[0]), int(item[1])))
        return ids
    
    tile_info = get_tile_info(layout, i, j)
    if tile_info is None:
        return None
    
    left_products = convert_items_to_int(tile_info['left_products'])
    right_products = convert_items_to_int(tile_info['right_products'])
    
    return [
        level,
        tile_info['dist_to_cashier'], 
        tile_info['dist_to_exit'], 
        tile_info['orientation'][0],
        tile_info['orientation'][1],
        left_products[0][0],
        left_products[0][1],
        left_products[1][0],
        left_products[1][1],
        left_products[2][0],
        left_products[2][1],
        left_products[3][0],
        left_products[3][1],
        right_products[0][0],
        right_products[0][1],
        right_products[1][0],
        right_products[1][1],
        right_products[2][0],
        right_products[2][1],
        right_products[3][0],
        right_products[3][1],
        tile_enum_to_int(layout[i-1][j]),
        tile_enum_to_int(layout[i+1][j]),
        tile_enum_to_int(layout[i][j-1]),
        tile_enum_to_int(layout[i][j+1]),
        tile_enum_to_int(layout[i-1][j-1]),
        tile_enum_to_int(layout[i+1][j+1]),
        tile_enum_to_int(layout[i-1][j+1]),
        tile_enum_to_int(layout[i+1][j-1]),
    ]

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
def custom_score(y_true, y_pred):
    return sum([y_true[i] * y_pred[i] * SCORE_COEFFICIENTS[i] for i in range(len(y_true))])

In [17]:
best = None